#Classifying Defensive Coverages with Convolutional Nueral Network
##By Spencer Kerch
###Credit and masssive thanks to Ben Baldwin for his Torch for R tutorial and his ngscleanr documentation for helping me through this 


For my Deep Learning and AI Ethics class taught by Dr. Scott Hawley, our final project was to build some sort of deep learning algorothim we had covered during the course and present it to the class. As a reader of Open Source Football I wanted to recreate the defensive coverage classifier from his [Computer Vision with NFL Player Tracking Data using torch for R](https://www.opensourcefootball.com/posts/2021-05-31-computer-vision-in-r-using-torch/) post. When I first read his blog post nearly all of it went over my head. However, after learning more about how nueral networks work from my class the proccess made much more sense. The classifier I made is more simple to reflect my skill and understanding of neural networks. Im am very proud of how the model turned out as this was easily one of the most difficult projects I've done in the time I have been doing statistical programming - a little over a year. If while reading this you have any feedback for me on my code or my process please contact me I'd love to hear it.

I first read in data I got from the NFL's [2021 Big Data Bowl](https://www.kaggle.com/c/nfl-big-data-bowl-2021/data) as well as established a classes list with only the names of the different coverages

In [1]:
import pandas as pd
coverages = pd.read_csv("/Users/spencerkerch/Desktop/proj/coverages_week1.csv")  #contains the coverage for each play in week 1(only week 1 data was provided for coverages)
plays = pd.read_csv("/Users/spencerkerch/Desktop/proj/plays.csv")
week1 = pd.read_csv("/Users/spencerkerch/Desktop/proj/week1.csv") #week 1 tracking data
games =  pd.read_csv("/Users/spencerkerch/Desktop/proj/games.csv")
classes = ["Cover 0 Man","Cover 1 Man","Cover 2 Man","Cover 2 Zone", "Cover 3 Zone", "Cover 4 Zone","Cover 6 Zone", "Prevent Zone"]

Now I filter the dataframes I created above to get the data I need to work with.

In [2]:
merged_df = pd.merge(week1,coverages, on = ["gameId","playId"] )
import numpy as np
#I dont need Football or QB data
merged_df = merged_df.query('displayName != "Football"')
merged_df = merged_df.query('position != "QB"')
#I will only use the tracking data for the ball snap frame to predict coverages
merged_df = merged_df.query('event == "ball_snap"')
#I need to distinguish defense from offense
merged_df.position.unique()
offense = ["RB","WR","FB","HB","TE"]
merged_df = merged_df.assign(defense = [0 if position in offense else 1 for position in merged_df['position']])
merged_df.head
merged_df = pd.merge(merged_df,plays,on=["gameId","playId"])

I understand that since coverages are often disguised pre-snap, using a singular frame at the snap of the ball will likely cause misclassifications. But for simplicity's sake I'm chosing to do this. * I'm also curious to see if the misclassifications can be used to help determine what looks teams like to fake *

Now I need to clean the data to get accurate values and create other values I will use in the model. In Ben's blog post he simply used his ngscleanr package to get most of this done; however, there appears to be no equivelant python package. So using math and the [ngscleanr documentation](https://github.com/guga31bb/ngscleanR/blob/master/R/cleaning_functions.R) I do the required calcualtions below.

In Bens model he includes orientation or more specifically orientation to the qb. I'll leave these out to make the calculations easier on me and save time. If I included these it would have no doubt benefited the model.

In [3]:
#changing x and y coords to be the same no matter the play direction
merged_df['x'] = np.where(merged_df['playDirection'] == 'left', 120 - merged_df['x'] , merged_df['x'])
merged_df['y'] = np.where(merged_df['playDirection'] == 'left', (160/3) - merged_df['y'] , merged_df['y'])
#approximation of the line of scrimmage
merged_df['los_x'] = np.where(merged_df['playDirection'] == 'left', 120 - merged_df['absoluteYardlineNumber'] , merged_df['absoluteYardlineNumber'])
#distance from line of scrimage
merged_df['dist_from_los'] = merged_df['x'] - merged_df['los_x']
#changing player direction to be the same no matter the play direction, as well converting to radians
merged_df['dir'] = np.where(merged_df['playDirection'] == 'left', merged_df['dir'] + 180 , merged_df['dir'])
merged_df['dir'] = np.where(merged_df['dir'] > 360,merged_df['dir'] - 360, merged_df['dir'])
merged_df['dir_rad'] = np.pi *(merged_df['dir']/180)
#direction in the x and y
merged_df['dir_x'] = np.sin(merged_df['dir_rad'])
merged_df['dir_y'] = np.cos(merged_df['dir_rad'])
#speed in the x and y
merged_df['s_x'] = merged_df['dir_x'] * merged_df['s']
merged_df['s_y'] = merged_df['dir_y'] * merged_df['s']
#acceleration in the x and y
merged_df['a_x'] = merged_df['dir_x'] * merged_df['a']
merged_df['a_y'] = merged_df['dir_y'] * merged_df['a']
#merged_df['play'] = str( merged_df['gameId']) + str(merged_df['playId'])

The next step is to seperate the offense and defese tracking data into 2 data frames. I simply filter around the defense variable I created earlier. I also rename any offense variable to o_variable to differentiate offense data from defense data.

In [4]:
offense_df = merged_df.query('defense == 0 & position != "QB"')
#print(offense_df['position'])
offense_df = offense_df.rename(columns={'x' : 'o_x', 'y' : 'o_y', 's_x':'o_s_x', 's_y':'o_s_y', 'a_x':'o_a_x', 'a_y':'o_a_y'})
#print(offense_df.head)
offense_df = offense_df[['gameId', 'playId', 'o_x', 'o_y', 'o_s_x', 'o_s_y','o_a_x','o_a_y']]
#print(offense_df.head)
#offense_df = offense_df.drop(offense_df.columns[[3,5]],axis=1)
print(offense_df.head)
defense_df = merged_df.query('defense == 1')
defense_df = defense_df[['gameId', 'playId','nflId', 'x', 'y', 's_x', 's_y','a_x','a_y', 'dist_from_los']]
print(defense_df)

<bound method NDFrame.head of            gameId  playId    o_x        o_y     o_s_x     o_s_y     o_a_x  \
1      2018090600      75  28.64   9.193333 -0.000000 -0.000000 -0.007645   
3      2018090600      75  29.22  17.173333  0.000000 -0.000000  0.000000   
5      2018090600      75  21.75  26.713333  0.000000  0.000000  0.000000   
9      2018090600      75  28.70  31.483333 -0.000000  0.000000 -0.005749   
11     2018090600      75  24.88  26.633333  0.000000  0.000000  0.000000   
...           ...     ...    ...        ...       ...       ...       ...   
13074  2018091001    3976  54.56   9.923333  0.314237 -0.060456  0.883793   
13076  2018091001    3976  55.29  36.183333  0.019148 -0.035119  0.019148   
13080  2018091001    3976  56.09  23.173333  0.089838  0.005400  0.099820   
13081  2018091001    3976  54.50  16.763333  0.400951 -0.085664  0.156469   
13086  2018091001    3976  51.69  32.233333  0.042254  0.055809  0.042254   

          o_a_y  
1     -0.006447  
3     -0.

Next I merge the offense and defense df into one df for each play.

In [5]:
df = defense_df.merge(offense_df, how = 'left', on = ['gameId','playId'])
print(df.head)

<bound method NDFrame.head of            gameId  playId      nflId      x          y       s_x       s_y  \
0      2018090600      75    79848.0  31.11  16.833333 -0.005941 -0.008044   
1      2018090600      75    79848.0  31.11  16.833333 -0.005941 -0.008044   
2      2018090600      75    79848.0  31.11  16.833333 -0.005941 -0.008044   
3      2018090600      75    79848.0  31.11  16.833333 -0.005941 -0.008044   
4      2018090600      75    79848.0  31.11  16.833333 -0.005941 -0.008044   
...           ...     ...        ...    ...        ...       ...       ...   
39718  2018091001    3976  2558183.0  66.55  32.223333  1.396686  0.193309   
39719  2018091001    3976  2558183.0  66.55  32.223333  1.396686  0.193309   
39720  2018091001    3976  2558183.0  66.55  32.223333  1.396686  0.193309   
39721  2018091001    3976  2558183.0  66.55  32.223333  1.396686  0.193309   
39722  2018091001    3976  2558183.0  66.55  32.223333  1.396686  0.193309   

            a_x       a_y  dist_f

Next, to get variables that are useful for the model I create "diff" variables for the difference in the offense and defense. This shows each defensive player in relation to each offensive player. I'll leave the qb out of the group of offensive players to only show the defender in relation to possible recievers, which is needed to determine coverage.

In [6]:
df['diff_x'] = df['o_x'] - df['x']
df['diff_y'] = df['o_y'] - df['y']
df['diff_s_x'] = df['o_s_x'] - df['s_x']
df['diff_s_y'] = df['o_s_y'] - df['s_y']
df['diff_a_x'] = df['o_a_x'] - df['a_x']
df['diff_a_y'] = df['o_a_y'] - df['a_y']
df = df[['gameId','playId','nflId','dist_from_los', 'y', 's_x', 's_y','a_x','a_y','diff_x','diff_y','diff_s_x','diff_s_y','diff_a_x','diff_a_y']]
df[['nflId','dist_from_los','y','diff_x',"diff_y"]].head


<bound method NDFrame.head of            nflId  dist_from_los          y  diff_x  diff_y
0        79848.0           1.11  16.833333   -2.47   -7.64
1        79848.0           1.11  16.833333   -1.89    0.34
2        79848.0           1.11  16.833333   -9.36    9.88
3        79848.0           1.11  16.833333   -2.41   14.65
4        79848.0           1.11  16.833333   -6.23    9.80
...          ...            ...        ...     ...     ...
39718  2558183.0           9.55  32.223333  -11.99  -22.30
39719  2558183.0           9.55  32.223333  -11.26    3.96
39720  2558183.0           9.55  32.223333  -10.46   -9.05
39721  2558183.0           9.55  32.223333  -12.05  -15.46
39722  2558183.0           9.55  32.223333  -14.86    0.01

[39723 rows x 5 columns]>

Now I need to create an index holding each play and it's respective coverage. Combining the gameId and playId for each play creates a unique identifier.

In [7]:
from sklearn.preprocessing import MinMaxScaler
play_index = plays.merge(coverages,on = ["gameId","playId"])
play_index['play'] = play_index['gameId'].map(str) + "_" + play_index['playId'].map(str)
play_index = play_index[['play', 'coverage']]
print(play_index)
n_plays = play_index['play'].size -1 #6 offense
print(n_plays)
df['play'] = df['gameId'].map(str) + "_" + df['playId'].map(str)
print(df)
df = df.drop(columns = ['gameId','playId'])
print(df)
play_index['i'] = play_index.index + 1
print(play_index)
#drop 479 due to nulls in that play
play_index = play_index.drop(index = 479)#6 offense

                 play      coverage
0       2018090600_75  Cover 3 Zone
1      2018090600_146  Cover 3 Zone
2      2018090600_168  Cover 3 Zone
3      2018090600_190  Cover 3 Zone
4      2018090600_256   Cover 0 Man
...               ...           ...
1023   2018091000_372   Cover 1 Man
1024  2018091000_2303   Cover 1 Man
1025  2018091000_2904  Cover 2 Zone
1026   2018091001_741  Cover 2 Zone
1027  2018091001_1181  Cover 4 Zone

[1028 rows x 2 columns]
1027
           gameId  playId      nflId  dist_from_los          y       s_x  \
0      2018090600      75    79848.0           1.11  16.833333 -0.005941   
1      2018090600      75    79848.0           1.11  16.833333 -0.005941   
2      2018090600      75    79848.0           1.11  16.833333 -0.005941   
3      2018090600      75    79848.0           1.11  16.833333 -0.005941   
4      2018090600      75    79848.0           1.11  16.833333 -0.005941   
...           ...     ...        ...            ...        ...       ...   
39718 

Thats it for data prep and cleaning! I'll admit I'm somewhat new to python especially compared to my experience with R and this part of the project did take a decent amount of time. But now I get to move on to more deep learning and nueral network stuff. It's now time to create and fill my tensors. A tensor is a torch object used in nueral networks that is a multidimensional matrix. I want the tensor to be 1027(number of plays)x12(number of features)x11(possible defenders on a play)x5(possible players on offense on a play). It is very hard for me to picture things once their past the 3rd dimension, but I choose to interpert this in my brain as for every play we use the 12 features for each defender in relation to each receiver on offense to predict the coverages. Anyways, I'll need to create an empty matrix of size 1027,12,11,5 so I'll use torch.zeros to fill a matrix that size all with 0.

In [8]:
!pip install torch torchvision
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
n_features = 12 #'dist_from_los', 'y', 's_x', 's_y','a_x','a_y','diff_x','diff_y','diff_s_x','diff_s_y','diff_a_x','diff_a_y'
data_tensor = torch.zeros((n_plays,n_features,11,5), dtype = torch.float32)
print(data_tensor.size())

torch.Size([1027, 12, 11, 5])


This function fills each tensor and does so by transposing the play_df into the right shape. Transposing flips the play so if it is size 35x12, it becomes 12x35. Making the variables the row and each instance of a defender in relation to the offense the collumns. The final line makes sure that when filling the tensor, it cuts off every 5, creating the shape I want of 12 7x5 matrices. Each matrix relating to a variable, each row relating to a defender, and each collumn relating to the defender in relation to the offense.

In [9]:
def fill_tensor(row):
    i = row
    row = play_index[row - 1:row]
    #i = row['i']
    playId = row.iloc[0]['play']
    play_df = df.query('play == @playId')
    play_df = play_df.drop(columns = 'play')
    #number of defenders
    defenders = pd.unique(play_df['nflId']).size
    #number of offense if there are 35 rows and 7 defenders that means there are 5 players on offense we are tracking. 35/7=5
    offense_players = play_df['nflId'].size / defenders
    play_df = play_df.drop(columns = 'nflId')
    t = play_df.T
    #Thanks to Ben and Corey in the nflfastr discord for teaching me about python indexing so I could fill the tensor
    data_tensor[i-1,:,:defenders,:int(offense_players)] = torch.from_numpy(t.values).view(-1,defenders,int(offense_players))
    

The loop below fills our tensor over all the 1027 plays

In [10]:


n = 1
while n <= n_plays:
    fill_tensor(n)
    n+=1

I still dont exactly know why, but some plays returned all nulls. Im sure it has to do with my code. I didnt have enough time to fix the problem as finals was approaching so I'll shorten the data to where I notice a pattern of every play being null began. Their were still num nan's here and there however, so I use nan_to_num to allow my network to run.

In [11]:
data_tensor = data_tensor[0:985]
print(data_tensor.shape)
data_tensor = torch.nan_to_num(data_tensor)
print(data_tensor.shape)

torch.Size([985, 12, 11, 5])
torch.Size([985, 12, 11, 5])


For my net to correctly classify coverages, I need to turn the coverages into a factor and turn it into a 1xn_plays sized tensor

In [12]:
import torch
import torch.nn.functional as F
play_index_factor = pd.factorize(play_index['coverage'])
print(play_index_factor)
#play_index_factor['coverage'] = play_index_factor['coverage'].astype(object)
play_index_factor = play_index_factor[0]
play_index_factor = play_index_factor.tolist()
coverage_tensor = torch.tensor(play_index_factor)
#coverage_tensor = F.one_hot(coverage_tensor)
#coverage_tensor = coverage_tensor.float()
print(coverage_tensor)
#print(F.one_hot(coverage_tensor,num_classes = 7))
#coverage_tensor = F.one_hot(coverage_tensor,num_classes = 7)

(array([0, 0, 0, ..., 5, 5, 4]), Index(['Cover 3 Zone', 'Cover 0 Man', 'Cover 1 Man', 'Cover 2 Man',
       'Cover 4 Zone', 'Cover 2 Zone', 'Cover 6 Zone'],
      dtype='object'))
tensor([0, 0, 0,  ..., 5, 5, 4])


Next, I seperate test and and training groups. I train on 785 plays and test on the other 200

In [13]:
import random as rand
test_size = 200
rand.seed(80085)
testId = rand.sample(range(985),k = test_size)
#print(testId)
test_data = data_tensor[testId]
print(test_data.size())
test_coverage = coverage_tensor[testId]
test_coverage = test_coverage
print(test_coverage)
trainId = list(set(range(985))-set(testId)) + list(set(testId)-set(range(985)))
#print(trainId)
train_data = data_tensor[trainId]
train_coverage = coverage_tensor[trainId]
train_coverage = train_coverage

#train_data=F.normalize(train_data)
#test_data=F.normalize(test_data)
print(train_coverage.size())

torch.Size([200, 12, 11, 5])
tensor([5, 4, 4, 0, 5, 2, 5, 0, 2, 0, 0, 5, 6, 3, 2, 0, 2, 4, 2, 0, 4, 5, 2, 0,
        0, 5, 4, 2, 2, 5, 5, 0, 2, 2, 4, 2, 5, 0, 4, 4, 2, 0, 2, 4, 5, 2, 0, 4,
        0, 2, 6, 5, 2, 2, 2, 0, 0, 5, 0, 5, 6, 0, 4, 0, 2, 3, 2, 0, 0, 2, 2, 0,
        2, 6, 4, 0, 4, 2, 4, 2, 3, 2, 2, 2, 0, 6, 2, 2, 4, 0, 0, 0, 0, 0, 0, 1,
        2, 5, 2, 2, 0, 2, 6, 4, 0, 5, 0, 4, 6, 5, 2, 0, 2, 2, 0, 4, 5, 0, 4, 0,
        0, 2, 6, 2, 2, 0, 0, 1, 4, 1, 2, 5, 0, 2, 6, 2, 0, 5, 0, 4, 4, 0, 0, 0,
        2, 5, 0, 2, 5, 4, 3, 2, 0, 5, 0, 5, 2, 4, 2, 0, 2, 0, 2, 6, 2, 0, 2, 0,
        4, 0, 0, 0, 2, 0, 0, 0, 2, 0, 2, 0, 3, 0, 0, 0, 2, 5, 0, 5, 3, 2, 5, 4,
        2, 0, 0, 3, 0, 2, 5, 4])
torch.Size([785])


It's time for me to set up the dataset and dataloader from torch. These allow the data to be put in a way that allows the network to train and predict the data.

In [14]:
from torch.utils.data import Dataset, DataLoader
class Dataset(Dataset):
    def __init__(self, x_tensor, y_tensor):
        self.data_y = y_tensor
        self.data_x = x_tensor
        

    def __len__(self):
        return int(self.data_y.size()[0])

    def __getitem__(self, idx):
        return self.data_x[idx,:], self.data_y[idx]


In [15]:
train_dataset = Dataset(train_data,train_coverage)

I set the batch size at 64, which yeilds 13 batches, since 64 goes into 785 13 times.(not evenly)

In [16]:
train_dataLoad = DataLoader(train_dataset, batch_size = 64, shuffle = True)
print(len(train_dataLoad))


13


Here is where I define the network. I want the network to to end with 7 outputs, one for each possible coverage. I use leaky_relu as opposed to relu mainly because I had time to. When I learned about it I liked how it didn't create dead nuerons. And it didn't make my model take outrageously long to run so I was able to use it. I compared it to relu and leaky relu did yeild bettter results. I use cross entropy loss as my loss citerion. The cross entropy loss function applies softmax to my data, turning the output tensor into the probability that the given play is each of the coverages. The highest probability is assigned as the prediction. 

In [17]:
print(pd.unique(play_index_factor).size)
n_coverage = pd.unique(play_index_factor).size
import torch.nn as nn
import torch.nn.functional as F

#convolutional!
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(12,128,1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(128,160,1)
        self.fc1 = nn.Linear(320, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, n_coverage)
    def forward(self,x):
        x = self.pool(F.leaky_relu(self.conv1(x)))
        x = self.pool(F.leaky_relu(self.conv2(x)))
        x = torch.flatten(x,1) 
        x = F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        x = self.fc3(x)
       
        #print(x)
        return x
    
net = Net()





7


I previously explained the reason for cross entropy as the loss criterion, and then for the optimizer I use a learning rate of .001

In [18]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

I chose 400 as the total amount of epochs, over many trial runs this seemed to be the best option, anything higher yielded little or no benefit.

In [19]:
torch.manual_seed(0)
for epoch in range(400):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_dataLoad):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 36== 0:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss ))
            running_loss = 0.0
#print(inputs, labels)
print('Finished Training')

[1,     1] loss: 1.907
[2,     1] loss: 1.690
[3,     1] loss: 1.694
[4,     1] loss: 1.553
[5,     1] loss: 1.560
[6,     1] loss: 1.444
[7,     1] loss: 1.622
[8,     1] loss: 1.561
[9,     1] loss: 1.431
[10,     1] loss: 1.581
[11,     1] loss: 1.589
[12,     1] loss: 1.482
[13,     1] loss: 1.500
[14,     1] loss: 1.447
[15,     1] loss: 1.496
[16,     1] loss: 1.554
[17,     1] loss: 1.513
[18,     1] loss: 1.386
[19,     1] loss: 1.407
[20,     1] loss: 1.436
[21,     1] loss: 1.617
[22,     1] loss: 1.372
[23,     1] loss: 1.589
[24,     1] loss: 1.689
[25,     1] loss: 1.438
[26,     1] loss: 1.516
[27,     1] loss: 1.505
[28,     1] loss: 1.645
[29,     1] loss: 1.188
[30,     1] loss: 1.541
[31,     1] loss: 1.527
[32,     1] loss: 1.557
[33,     1] loss: 1.352
[34,     1] loss: 1.420
[35,     1] loss: 1.308
[36,     1] loss: 1.398
[37,     1] loss: 1.352
[38,     1] loss: 1.436
[39,     1] loss: 1.397
[40,     1] loss: 1.277
[41,     1] loss: 1.588
[42,     1] loss: 1.376
[

[334,     1] loss: 0.044
[335,     1] loss: 0.063
[336,     1] loss: 0.053
[337,     1] loss: 0.049
[338,     1] loss: 0.046
[339,     1] loss: 0.059
[340,     1] loss: 0.044
[341,     1] loss: 0.036
[342,     1] loss: 0.034
[343,     1] loss: 0.050
[344,     1] loss: 0.049
[345,     1] loss: 0.037
[346,     1] loss: 0.046
[347,     1] loss: 0.052
[348,     1] loss: 0.044
[349,     1] loss: 0.045
[350,     1] loss: 0.050
[351,     1] loss: 0.033
[352,     1] loss: 0.038
[353,     1] loss: 0.041
[354,     1] loss: 0.035
[355,     1] loss: 0.037
[356,     1] loss: 0.047
[357,     1] loss: 0.035
[358,     1] loss: 0.037
[359,     1] loss: 0.059
[360,     1] loss: 0.074
[361,     1] loss: 0.037
[362,     1] loss: 0.030
[363,     1] loss: 0.043
[364,     1] loss: 0.024
[365,     1] loss: 0.024
[366,     1] loss: 0.046
[367,     1] loss: 0.031
[368,     1] loss: 0.024
[369,     1] loss: 0.033
[370,     1] loss: 0.022
[371,     1] loss: 0.028
[372,     1] loss: 0.030
[373,     1] loss: 0.020


In [20]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

In [21]:
net = Net()
net.load_state_dict(torch.load(PATH))
test_dataset = Dataset(test_data,test_coverage)
test_dataLoad = DataLoader(test_dataset, batch_size = 64, shuffle = True)
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in test_dataLoad:
        plays, labels = data
        
        # calculate outputs by running images through the network
        outputs = net(plays)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        print(predicted.shape,labels.shape)
        correct += (predicted == labels).sum().item()
        print(labels)
        print(predicted)
        
print('Accuracy of the network on the 985 test plays: %d %%' % (
    100 * correct / total))



torch.Size([64]) torch.Size([64])
tensor([0, 0, 2, 0, 0, 2, 6, 2, 2, 2, 2, 0, 5, 4, 5, 0, 0, 5, 2, 2, 0, 0, 2, 2,
        0, 4, 4, 0, 0, 5, 4, 5, 5, 4, 0, 2, 0, 3, 4, 6, 0, 2, 0, 5, 0, 2, 0, 0,
        0, 2, 2, 5, 6, 0, 2, 0, 4, 3, 0, 5, 0, 5, 2, 4])
tensor([0, 0, 2, 0, 4, 0, 4, 2, 2, 2, 0, 0, 2, 0, 2, 0, 0, 6, 2, 2, 0, 3, 0, 2,
        0, 5, 0, 4, 0, 5, 0, 0, 5, 4, 0, 4, 0, 5, 4, 0, 0, 5, 2, 5, 2, 0, 0, 0,
        0, 2, 2, 0, 4, 0, 2, 2, 4, 2, 0, 4, 4, 2, 2, 4])
torch.Size([64]) torch.Size([64])
tensor([2, 2, 0, 4, 2, 0, 4, 0, 2, 3, 2, 2, 5, 6, 4, 3, 3, 4, 4, 4, 2, 3, 2, 5,
        1, 5, 2, 6, 0, 0, 0, 0, 2, 0, 4, 5, 0, 2, 0, 2, 2, 0, 0, 0, 0, 0, 0, 2,
        0, 2, 0, 4, 0, 2, 2, 2, 2, 4, 3, 4, 0, 2, 2, 2])
tensor([2, 2, 0, 4, 2, 2, 0, 4, 6, 2, 2, 2, 5, 5, 6, 3, 2, 0, 0, 5, 2, 2, 2, 5,
        2, 5, 2, 5, 0, 5, 0, 0, 2, 0, 4, 2, 0, 0, 0, 2, 3, 2, 0, 0, 4, 5, 0, 2,
        4, 2, 0, 4, 0, 2, 5, 2, 2, 4, 4, 0, 2, 2, 2, 2])
torch.Size([64]) torch.Size([64])
tensor([2, 0, 2, 2, 2, 2, 4, 0

For my first neural network I am very happy with what I made. While 53% doesn't scream success it is greater than random, or 1/7. Ben's model had 78% accuracy, and as I learn more about neural networks and how to build them, along with using more frames over the course of the play and including orientation to the qb, I believe I could improve my model greatly.